In [1]:
import numpy as np

In [2]:
num_sample = 500

data0 = np.random.randn(num_sample,2) + (2,2)
data1 = np.random.randn(num_sample,2) + (-2,-2)

data0 = np.hstack([data0,np.zeros((num_sample,1),dtype=float)])
data1 = np.hstack([data1,np.ones((num_sample,1),dtype=float)])

data = np.vstack([data0,data1]) # 두 라벨의 데이터를 쭉 쌓음 

print(data.shape)

(1000, 3)


In [3]:
data0.shape

(500, 3)

In [9]:
# we are assuming one layer logistic regression
# 학습 파라미터의 개수 
w = np.random.randn(2,1)
b = np.random.randn(1,1)
eta = 1e-5 # learning rate
delta = 1e-10 # prevent log 0

In [5]:
def sigmoid(val):
    return 1 / (1 + np.exp(-val))

def grad_sigmoid(val):
    return sigmoid(val) * (1 - sigmoid(val))

def loss_ftn(pred, target, delta=1e-10):
    pred = np.clip(pred, delta, 1 - delta) # delta로 계산 폭주 방지 
    return -target*np.log(pred) - (1 - target)*np.log(1 - pred)

def grad_loss_ftn(pred, target):
    return -(target / pred) + ((1 - target) / (1 - pred))

def compute_loss_and_grad(data_instance):
    """
    나는 이걸 단계별로 구현했는데, chain-rule을 이용해 미분한 식을 만든 뒤에 값을 다 넣어도 됨 
    ㄴ 당연한거 아닌가요?
    ㄴ 헷갈릴 수도 있지. 
    """
    x, y = data_instance

    # forward pass
    h = x @ w + b
    pred = sigmoid(h)
    loss = loss_ftn(pred, y)

    # backward pass 
    grad_loss = grad_loss_ftn(pred, y)
    grad_sig = grad_sigmoid(h)

    grad_w = (grad_loss * grad_sig).T @ x  # (1,1) * (1,2) -> (1,2)
    grad_b = np.sum(grad_loss * grad_sig)  # (1,1) -> scalar

    # 정답 예측 
    hit = (np.round(pred) == y).astype(int)

    return loss, (grad_w.T, grad_b), hit

def update_parameters(params, grads):
    w, b = params
    grad_w, grad_b = grads

    w -= eta * grad_w  # (2,1)
    b -= eta * grad_b  # (1,1)


In [10]:
num_epoch = 100

for i in range(num_epoch):
    perm = np.random.permutation(len(data))
    data = data[perm]

    total_loss = 0
    count = 0

    for j in range(len(data)):
        x = data[j][:-1].reshape(1, -1)
        y = data[j][-1]
        y = np.array(y).reshape(1, 1)

        params = (w, b)
        loss, grads, n_correct = compute_loss_and_grad((x, y))
        total_loss += loss
        count += n_correct
        update_parameters(params, grads)

    loss_train = total_loss / len(data)
    acc_train = (count / len(data)) * 100

    if i % 5 == 0:
        print("Epoch %d Train: %.3f / %.2f %%" % (i, loss_train, acc_train))

/var/folders/zl/ljg24f_15m93w7f2rt14c2n80000gn/T/ipykernel_86376/2644411146.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Epoch %d Train: %.3f / %.2f %%" % (i, loss_train, acc_train))


Epoch 0 Train: 1.050 / 33.90 %
Epoch 5 Train: 0.921 / 42.70 %
Epoch 10 Train: 0.813 / 50.40 %
Epoch 15 Train: 0.723 / 56.70 %
Epoch 20 Train: 0.647 / 63.20 %
Epoch 25 Train: 0.582 / 67.90 %
Epoch 30 Train: 0.528 / 72.50 %
Epoch 35 Train: 0.482 / 76.40 %
Epoch 40 Train: 0.442 / 80.20 %
Epoch 45 Train: 0.408 / 83.90 %
Epoch 50 Train: 0.379 / 85.80 %
Epoch 55 Train: 0.353 / 88.10 %
Epoch 60 Train: 0.330 / 89.60 %
Epoch 65 Train: 0.310 / 90.90 %
Epoch 70 Train: 0.293 / 92.50 %
Epoch 75 Train: 0.277 / 93.00 %
Epoch 80 Train: 0.263 / 94.10 %
Epoch 85 Train: 0.250 / 94.50 %
Epoch 90 Train: 0.238 / 95.10 %
Epoch 95 Train: 0.228 / 95.60 %


반복횟수, lr와 같은 지표를 성능을 올리기 위해 고민해보기  

근데 그거 생각해도 많이 큰데 